In [1]:
!pip3 install PyPDF2
!pip install langchain
!pip install -qU langchain-groq
!pip install langchain_community
!pip install rouge-score
!pip install transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import PyPDF2
import os
import ast
from google.colab import userdata
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

In [3]:
!pip install PyCryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.6 MB/s eta 0:00:00


In [4]:
import PyPDF2


report = ""

# Open the PDF file
with open('/content/Sample-filled-in-MR.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)
    print(f"The PDF has {num_pages} pages.")

    # Extract the text content from each page
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        report += text

print(report)

The PDF has 10 pages.
- 1 - 
 SAMPLE  
(All names and details provided in this sample are fictitious.   
Some fields have been deliberately left blank. ) 
 
 
MEDICAL REPORT  
 
SECTION 1: PATIENT’S PARTICULARS  
 
Full name of patient:  Mr Tan Ah Kow  
 
NRIC/FIN/Passport no. of patient:  S1111111X  
 
Age of patient:  55 years old  
 
SECTION 2: DOCTOR’S PARTICULARS  
 
Full name of doctor:  Tan Ah Moi  
 
NRIC/FIN/Passport no. of doctor:  S2222222Z   
 
 
MCR no. of doctor:  333333  
 
Hospital / Clinic name and address:  1 Blackacre Hospital, Singapore 01010101  
 
 
 
Doctor’s qualifications and experience in this area of work:  
 
 
[To set out details ] 
 
 
 
 
 
 
 - 2 - 
  
Doctor -patient relationship:  
 
Please state if you have been seeing the patient regularly over a period of time (if so, 
please state when you first started see ing the patient and how often you see the 
patient) or if you saw the patient specifically for this mental capacity assessment only.  
 
 
I ha

In [5]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [6]:
!pip install langchain_groq

In [7]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-70b-versatile", max_tokens=8000)

In [8]:
import ast
from rouge_score import rouge_scorer

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage

messages = [
    SystemMessage(content="""You have to summarize the Medical report.
    You should be able to specify the important details specifically like patient age, the complaint and the diseases suffered by the patient.
    The doctor who reads the summary should able to remember the diagnosis done to the patient previously.

    Example JSON:

    {{
        "name" : Name of the patient,
        "age" : Age of the patient
        "prescription" : Medicines prescribed by the doctor,
        "Gender": Gender of the patient.
        "Medical History": Brief overview of the patient's relevant medical history.
        "Allergies": Any known allergies the patient has.
        "Symptoms": Key symptoms presented by the patient.
        "Vital Signs": Basic vital signs like blood pressure, heart rate, temperature, etc.
        "Tests Conducted" : List of tests conducted, if any, and their results.
        "Treatment Plan" : Outline of the treatment plan.
        "Follow-Up" : Scheduled follow-up date or required next steps.
        "Contact Information" : Patient's contact details for easy reference.
        "Doctor's Notes" : Additional observations or important notes by the doctor.
         "Date of Report" : Date when the report was created or updated.
        "short_summary" : Short summary of the report,
        "diagnosis" : Diagnosis done to the patient
    }}

    Do not start with Here is the summary....

    You have to output a STRICTLTY JSON ONLY.
    Do not output anything else. NO MARKDOWN JUST A STRING
    """),
    HumanMessage(content=f"{report}"),
]

summary = model.invoke(messages)

In [10]:
summary_d = summary.content

In [11]:
import ast
summary_d = summary_d.strip("```")
summary_dict = ast.literal_eval(summary_d)
summary_dict

{'name': 'Mr Tan Ah Kow',
 'age': 55,
 'prescription': '',
 'Gender': '',
 'Medical History': 'Hypertension, hyperlipidemia, stroke, heart problems (cardiomyopathy), cardiac failure, chronic renal disease, dementia',
 'Allergies': '',
 'Symptoms': 'Incontinence, unable to bathe or use the toilet on his own, unable to remember basic information, unable to perform simple arithmetic',
 'Vital Signs': '',
 'Tests Conducted': 'CT brain scan, MRI brain scan',
 'Treatment Plan': '',
 'Follow-Up': '',
 'Contact Information': '',
 "Doctor's Notes": 'Patient has dementia and stroke, which have impaired the functioning of his mind and brain. He is unable to understand, retain, use or weigh simple information, and is unable to make decisions about his personal and financial affairs.',
 'Date of Report': '20 July 2015',
 'short_summary': 'Patient has dementia and stroke, and lacks mental capacity to make decisions about his personal and financial affairs.',
 'diagnosis': 'Dementia, Stroke'}

In [12]:
# Define the reference summary
reference_summary = {
    "name": "Alice Smith",
    "age": 52,
    "prescription": "Some medication",
    "Gender": "Female",
    "Medical History": "Heart condition",
    "Allergies": "None mentioned",
    "Symptoms": "Cough",
    "Vital Signs": "Stable",
    "Tests Conducted": "General tests",
    "Treatment Plan": "Basic care",
    "Follow-Up": "Check-up in a week",
    "Contact Information": "Provided",
    "Doctor's Notes": "No significant concerns",
    "Date of Report": "2023-08-11",
    "short_summary": "Patient with a heart condition and a cough. Given basic medication.",
    "diagnosis": "Common cold"
}

# Convert both summaries to strings for ROUGE evaluation
generated_summary = str(summary_dict)
reference_summary_str = str(reference_summary)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(reference_summary_str, generated_summary)

# Display ROUGE scores
print("ROUGE-1:", scores['rouge1'])
print("ROUGE-2:", scores['rouge2'])
print("ROUGE-L:", scores['rougeL'])


ROUGE-1: Score(precision=0.23484848484848486, recall=0.45588235294117646, fmeasure=0.31000000000000005)
ROUGE-2: Score(precision=0.0916030534351145, recall=0.1791044776119403, fmeasure=0.1212121212121212)
ROUGE-L: Score(precision=0.22727272727272727, recall=0.4411764705882353, fmeasure=0.3)


In [15]:
import pickle

# Save the model and its configuration
model_config = {
    'model_name': "llama-3.1-70b-versatile",
    'max_tokens': 8000,
    'summary_prompt': """You have to summarize the Medical report.
        You should be able to specify the important details specifically like patient age, the complaint and the diseases suffered by the patient.
        The doctor who reads the summary should able to remember the diagnosis done to the patient previously.

        Example JSON:

        {{
            "name" : Name of the patient,
            "age" : Age of the patient,
            "prescription" : Medicines prescribed by the doctor,
            "Gender": Gender of the patient,
            "Medical History": Brief overview of the patient's relevant medical history,
            "Allergies": Any known allergies the patient has,
            "Symptoms": Key symptoms presented by the patient,
            "Vital Signs": Basic vital signs like blood pressure, heart rate, temperature, etc.,
            "Tests Conducted" : List of tests conducted, if any, and their results,
            "Treatment Plan" : Outline of the treatment plan,
            "Follow-Up" : Scheduled follow-up date or required next steps,
            "Contact Information" : Patient's contact details for easy reference,
            "Doctor's Notes" : Additional observations or important notes by the doctor,
            "Date of Report" : Date when the report was created or updated,
            "short_summary" : Short summary of the report,
            "diagnosis" : Diagnosis done to the patient
        }}

        Do not start with Here is the summary....

        You have to output a STRICTLY JSON ONLY.
        Do not output anything else. NO MARKDOWN JUST A STRING
    """
}

# Save the configuration
with open('model_config.pkl', 'wb') as config_file:
    pickle.dump(model_config, config_file)

print("Model configuration saved successfully.")


Model configuration saved successfully.


In [14]:
model.save_pretrained('/content/summarization_model')

AttributeError: 'ChatGroq' object has no attribute 'save_pretrained'